In [29]:
!npm install localtunnel
!pip install groq
!pip install streamlit pandas gspread  oauth2client google-auth google-auth-oauthlib google-auth-httplib2 langchain groq pydantic
!pip install requests
!pip install python-dotenv
!pip install streamlit





up to date, audited 23 packages in 484ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [30]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from groq import Groq  # Import Groq Client for LLM queries

# Load environment variables
load_dotenv()

# Set up API keys
scraperapi_key = os.getenv("SCRAPERAPI_KEY")  # Replace with your ScraperAPI key

st.title("AI Information Retrieval Dashboard")

# Step 1: Upload CSV file and display columns
st.header("Upload a CSV file")
uploaded_file = st.file_uploader("Choose a CSV file", type=["csv"])

if uploaded_file:
    data = pd.read_csv(uploaded_file)
    st.write("Data Preview:")
    st.write(data)
    columns = data.columns.tolist()
    primary_column = st.selectbox("Select the main column for entity extraction:", columns)
    st.write(f"Selected primary column: {primary_column}")

    # Step 2: Define Custom Prompt for Information Retrieval
    st.header("Define Search Query")
    st.write("Use placeholders like `{entity}` for the chosen column's values in your query.")
    custom_prompt = st.text_input("Enter your query prompt:", value="Retrieve email address details of {entity}")

    # Step 3: Function to Run ScraperAPI Project and Retrieve Data
    def run_scraperapi_query(query):
        # ScraperAPI endpoint to retrieve data
        url = f"http://api.scraperapi.com?api_key={scraperapi_key}&url={query}"
        response = requests.get(url)
        if response.status_code == 200:
            return response.text  # This will return the HTML or data from the URL
        else:
            return "Error retrieving data"

    # Step 4: Perform Web Search & Retrieve Information with ScraperAPI
    st.header("Retrieve Data from Web")
    if custom_prompt and primary_column:
        if st.button("Run Search"):
            results = []

            # Web search for each entity using ScraperAPI
            for entity in data[primary_column]:
                query = custom_prompt.replace("{entity}", entity)
                scraper_data = run_scraperapi_query(query)

                # Collect the raw web data for each entity
                results.append({"Entity": entity, "Web Data": scraper_data})

            # Display search results in a table
            results_df = pd.DataFrame(results)
            st.write("Raw Web Data:")
            st.write(results_df)

            # Step 5: Parse the Web Results Using Groq LLM
            client = Groq()  # Initialize Groq Client

            def query_groq_llm(prompt):
                completion = client.chat.completions.create(
                    model="llama3-70b-8192",
                    messages=[
                        {"role": "user", "content": prompt}
                    ],
                    temperature=1,
                    max_tokens=1024,
                    top_p=1,
                    stream=True
                )
                return "".join([chunk.choices[0].delta.content or "" for chunk in completion])

            # Extract specific info for each entity using Groq LLM
            extracted_data = []
            for entity, web_data in zip(data[primary_column], results_df["Web Data"]):
                prompt = f"Extract the email and address for {entity} from the following web data: {web_data}"
                result = query_groq_llm(prompt)
                extracted_data.append({"Entity": entity, "Extracted Info": result})

            extracted_df = pd.DataFrame(extracted_data)
            st.write("Extracted Information:")
            st.write(extracted_df)

            # Step 6: Option to Download Extracted Data as CSV
            st.download_button("Download Extracted Data as CSV", extracted_df.to_csv(index=False), "extracted_data.csv")

import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

!streamlit run app.py &>/content/logs.txt &

!npx localtunnel --port 8501

Overwriting app.py
